In [ ]:
import json
from collections import defaultdict
from datetime import datetime
import statistics

# Load the scraped message data
with open("messages.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# Load auto-labeled NER data
with open("auto_labeled.json", "r", encoding="utf-8") as f:
    labeled_data = json.load(f)

In [ ]:
combined_data = []
for meta, ner in zip(raw_data, labeled_data):
    combined_data.append({
        "channel": meta["channel"],
        "username": meta["username"],
        "timestamp": meta["timestamp"],
        "views": meta.get("views", 0),
        "tokens": ner["tokens"],
        "ner_tags": ner["ner_tags"]
    })

In [ ]:
vendors = defaultdict(list)

# Group by vendor
for entry in combined_data:
    vendors[entry["username"]].append(entry)

def extract_price(tokens, tags):
    prices = []
    for token, tag in zip(tokens, tags):
        if tag == "PRICE":
            token_clean = token.replace("ብር", "").replace("ETB", "").replace("Birr", "")
            try:
                prices.append(float(token_clean))
            except:
                pass
    return prices

vendor_stats = {}

for vendor, posts in vendors.items():
    views = []
    prices = []
    timestamps = []

    for post in posts:
        views.append(post.get("views", 0))
        timestamps.append(datetime.fromisoformat(post["timestamp"]))
        prices.extend(extract_price(post["tokens"], post["ner_tags"]))

    # Posting frequency
    days = (max(timestamps) - min(timestamps)).days + 1
    weeks = max(1, days / 7)
    posts_per_week = round(len(posts) / weeks, 2)

    avg_views = round(statistics.mean(views), 2) if views else 0
    avg_price = round(statistics.mean(prices), 2) if prices else 0
    top_post_index = views.index(max(views)) if views else -1
    top_product = " ".join([
        tok for tok, tag in zip(posts[top_post_index]["tokens"], posts[top_post_index]["ner_tags"])
        if tag == "PRODUCT"
    ]) if top_post_index != -1 else "N/A"

    top_price = next((tok for tok, tag in zip(posts[top_post_index]["tokens"], posts[top_post_index]["ner_tags"])
                     if tag == "PRICE"), "N/A") if top_post_index != -1 else "N/A"

    # Lending Score (example: 50% views, 50% posting activity)
    lending_score = round((avg_views * 0.5) + (posts_per_week * 0.5), 2)

    vendor_stats[vendor] = {
        "channel": posts[0]["channel"],
        "avg_views": avg_views,
        "posts_per_week": posts_per_week,
        "avg_price": avg_price,
        "top_product": top_product,
        "top_price": top_price,
        "lending_score": lending_score
    }

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(vendor_stats, orient="index")
df.reset_index(inplace=True)
df.rename(columns={"index": "Username"}, inplace=True)
df = df[["channel", "Username", "avg_views", "posts_per_week", "avg_price", "lending_score"]]
df.columns = ["Channel Name", "Username", "Avg Views/Post", "Posts/Week", "Avg Price (ETB)", "Lending Score"]

df.to_csv("vendor_scorecard.csv", index=False)
print(df)
